In [1]:
from PIL import Image, ImageStat
import pandas as pd
import os 
import matplotlib.pyplot as plt
import matplotlib.image as img
import numpy as np
import cv2
import statistics
import math
import imageio
from numba import njit
import re
import sys
from time import sleep
from tqdm import tqdm
import shutil
#%matplotlib qt

In [2]:
os.makedirs("frames", exist_ok=True)
os.makedirs("normalized", exist_ok=True)

def listdir_nohidden(path):
    for f in os.listdir(path):
        if not f.startswith('.'):
            yield f

# Read csv and make list of frames
def frame_list(csv):
    overlap = pd.read_csv(csv)
    frame_list = []
    for i in overlap["image"]:
        frame_list.append(i)
    overlap_indexed = overlap.set_index("image")
    return frame_list


# Gather information on each frame in a sections overlapping regions from a given hole
# and store in dictionaries
def make_dictionaries(hole, csv):
    overlap = pd.read_csv(csv)
    overlap_indexed = overlap.set_index("image")
    section_list = listdir_nohidden(hole)
    im_dict = {}
    cropped_im_dict = {}
    plt_dict = {}
    overlap_dict = {}
    frame_dict = {} 
    frame_overlap = {}
    
    for section in section_list:
        path = hole + "/" + section
        cropped_path = hole + "_cropped" + "/" + section
        image_frames = sorted(listdir_nohidden(path))
        section_list = listdir_nohidden(hole)

        image_frames = sorted(listdir_nohidden(path))#sorted to ensure frames are stored in order
        for frame in image_frames:
            
            if frame[-5] == "x" and frame.endswith("BMP") and frame in frame_list:
                frame_overlap[frame] = {}
                im_dict[frame] = Image.open(path+"/"+frame)
                cropped_im_dict[frame] = Image.open(cropped_path+"/"+frame)
                plt_dict[frame] = img.imread(path+"/"+frame)
                left_pixel_a = str(frame[-6])+"A"+"_left"
                right_pixel_a = str(frame[-6])+"A"+"_right"
                left_pixel_b = str(frame[-6])+"B"+"_left"
                right_pixel_b = str(frame[-6])+"B"+"_right"

                if str(frame[-6])+"A" == "1A":
                    overlap_list = (overlap_indexed.at[frame,left_pixel_b], 
                                    overlap_indexed.at[frame,right_pixel_b]) 
                    overlap_dict[str(frame[-6]+"B")] = overlap_list
                    frame_dict[frame] = ((frame[-6]+"B"))
                    frame_overlap[frame][frame[-6]+"B"] = overlap_list

                else: 
                    overlap_list_a = (overlap_indexed.at[frame,left_pixel_a], 
                                      overlap_indexed.at[frame,right_pixel_a])
                    overlap_list_b = (overlap_indexed.at[frame,left_pixel_b], 
                                      overlap_indexed.at[frame,right_pixel_b]) 
                    overlap_dict[str(frame[-6]+"A")] = overlap_list_a
                    overlap_dict[str(frame[-6]+"B")] = overlap_list_b
                    frame_dict[frame] = (frame[-6]+"A", frame[-6]+"B")
                    frame_overlap[frame][frame[-6]+"A"] = overlap_list_a
                    frame_overlap[frame][frame[-6]+"B"] = overlap_list_b    

    return im_dict, cropped_im_dict, plt_dict, overlap_dict, frame_dict, frame_overlap

# Crop out each frames overlappoing regions and save to ./frames 
def crop_overlaps(list_of_images, overlap_info, dict_of_img_objects):
    for image in frame_dict:
        im = dict_of_img_objects[image]
        frame_px_array = np.array(im)
        width, height = im.size
        
        for label in overlap_info[image]:
            if math.isnan(overlap_info[image][label][0]) == False:
                left = int(overlap_info[image][label][0])
                right = int(overlap_info[image][label][1]) 
            else:
                continue

            top = 10
            bottom = height - 10

            im1 = im.crop((left, top, right, bottom))
            A1 = im1
            A1.save("frames/"+image+"_"+label+".jpg")

# Calculate the mean and std dev of pixels in each band of the cropped overlapped images
def image_stats(folder_of_overlaps):
    stat_dict = {}
    for image in listdir_nohidden(folder_of_overlaps):
        stat_dict[image] = {}
        im = Image.open(folder_of_overlaps+"/"+image)
        width, height = im.size
        pxl_data = im.getdata()
        pxl_array = np.array(im)
        stat = ImageStat.Stat(im)
        mean = stat.mean #list of mean for each band
        stdv = stat.stddev #list of stdev for each band
        stat_dict[image]["Rmean"] = mean[0]
        stat_dict[image]["Bmean"] = mean[1]
        stat_dict[image]["Gmean"] = mean[2]
        stat_dict[image]["Rstdv"] = stdv[0]
        stat_dict[image]["Bstdv"] = stdv[1]
        stat_dict[image]["Gstdv"] = stdv[2]
    
    sorted_list = sorted(stat_dict.items())
    sorted_dict = dict(sorted_list)
    
    return sorted_dict


def divide_by_min(lst):
    smallest = min(lst)
    result = [x / smallest for x in lst]
    return result


def add_min(lst):
    smallest = min(lst)
    result = [x + abs(smallest) for x in lst]
    return result


def adjust_gain_offset(dataframe):
    corrected = dataframe.copy()
    corrected["piece"] = ""
    
    for i in range(len(corrected)):
        string = corrected.loc[i,"index"]
        match = re.match(r"([^-]+-[^-]+-[^-]+-[^-]+-[^-]+)-", string)
        extracted_string = match.group(1)
        corrected.loc[i, "piece"] = extracted_string
    
    grouped = corrected.groupby("piece").agg(list).reset_index("piece")
    grouped = grouped.copy()
    
    for i in range(len(grouped)):
        if min(grouped.loc[i,"Rgain"]) < 1:
            corrected_Rgain = divide_by_min(grouped.loc[i,"Rgain"])
            grouped.loc[i,"Rgain"] = corrected_Rgain 

        if min(grouped.loc[i,"Ggain"]) < 1:
            corrected_Rgain = divide_by_min(grouped.loc[i,"Ggain"])
            grouped.loc[i,"Ggain"] = corrected_Rgain 

        if min(grouped.loc[i,"Bgain"]) < 1:
            corrected_Rgain = divide_by_min(grouped.loc[i,"Bgain"])
            grouped.loc[i,"Bgain"] = corrected_Rgain 

        if min(grouped.loc[i,"Roffset"]) < 0:
            corrected_Rgain = add_min(grouped.loc[i,"Roffset"])
            grouped.loc[i,"Roffset"] = corrected_Rgain 

        if min(grouped.loc[i,"Goffset"]) < 0:
            corrected_Rgain = add_min(grouped.loc[i,"Goffset"])
            grouped.loc[i,"Goffset"] = corrected_Rgain 

        if min(grouped.loc[i,"Boffset"]) < 0:
            corrected_Rgain = add_min(grouped.loc[i,"Boffset"])
            grouped.loc[i,"Boffset"] = corrected_Rgain 
    
    return grouped

    
def gain_offset(image_stats):
    new = pd.DataFrame.from_dict(image_stats, orient="index").reset_index()
    new["Rgain"] = ""
    new["Ggain"] = ""
    new["Bgain"] = ""
    new["Roffset"] = ""
    new["Goffset"] = ""
    new["Boffset"] = ""

    for i in range(len(new)):
        print(i)
        if "1B" in new.loc[i,"index"]:
            new.loc[i, "Rgain"] = 1
            new.loc[i, "Ggain"] = 1
            new.loc[i, "Bgain"] = 1
            new.loc[i, "Roffset"] = 0
            new.loc[i, "Goffset"] = 0
            new.loc[i, "Boffset"] = 0

        elif "B.jpg" in new.loc[i,"index"]:
            new.loc[i, "Rgain"] = None
            new.loc[i, "Ggain"] = None
            new.loc[i, "Bgain"] = None
            new.loc[i, "Roffset"] = None
            new.loc[i, "Goffset"] = None
            new.loc[i, "Boffset"] = None
            
        
        elif "2A" in new.loc[i,"index"]:
            print(i)
            print(new.loc[i-1])
            print(new.loc[i])
            new.loc[i,"Rgain"] = (new.loc[i-1,"Rstdv"]/new.loc[i,"Rstdv"])*new.loc[i-1,"Rgain"]
            new.loc[i,"Ggain"] = (new.loc[i-1,"Gstdv"]/new.loc[i,"Gstdv"])*new.loc[i-1,"Ggain"]
            new.loc[i,"Bgain"] = (new.loc[i-1,"Bstdv"]/new.loc[i,"Bstdv"])*new.loc[i-1,"Bgain"]
            new.loc[i,"Roffset"] = (new.loc[i-1,"Rmean"]) - (new.loc[i,"Rgain"])*new.loc[i,"Rmean"]
            new.loc[i,"Goffset"] = (new.loc[i-1,"Gmean"]) - (new.loc[i,"Ggain"])*new.loc[i,"Gmean"]
            new.loc[i,"Boffset"] = (new.loc[i-1,"Bmean"]) - (new.loc[i,"Bgain"])*new.loc[i,"Bmean"]

        elif "A" in new.loc[i,"index"]:
            new.loc[i,"Rgain"] = (new.loc[i-1,"Rstdv"]/new.loc[i,"Rstdv"])*new.loc[i-2,"Rgain"]
            new.loc[i,"Ggain"] = (new.loc[i-1,"Gstdv"]/new.loc[i,"Gstdv"])*new.loc[i-2,"Ggain"]
            new.loc[i,"Bgain"] = (new.loc[i-1,"Bstdv"]/new.loc[i,"Bstdv"])*new.loc[i-2,"Bgain"]
            new.loc[i,"Roffset"] = (new.loc[i-1,"Rmean"]) - ((new.loc[i-1,"Rstdv"]/new.loc[i,"Rstdv"]))*new.loc[i,"Rmean"]*new.loc[i-2,"Rgain"]+new.loc[i-2,"Roffset"]
            new.loc[i,"Goffset"] = (new.loc[i-1,"Gmean"]) - ((new.loc[i-1,"Gstdv"]/new.loc[i,"Gstdv"]))*new.loc[i,"Gmean"]*new.loc[i-2,"Ggain"]+new.loc[i-2,"Goffset"]
            new.loc[i,"Boffset"] = (new.loc[i-1,"Bmean"]) - (new.loc[i-1,"Bstdv"]/new.loc[i,"Bstdv"])*new.loc[i,"Bmean"]*new.loc[i-2,"Bgain"]+new.loc[i-2,"Boffset"]
    
    new.dropna(axis=0, inplace=True)
    new.reset_index(drop=True, inplace=True)
    new['index'] = new['index'].str.replace(r'\.BMP.*$', '.BMP', regex=True)
    
    return new

"""@njit           
def adjust_pixels(image_array, Rgain, Ggain, Bgain, Roffset, Goffset, Boffset):
    #print(Rgain, Ggain, Bgain, Roffset, Goffset, Boffset)
    adjusted_array = image_array
    for column in range(len(adjusted_array)):
        c = adjusted_array[column]
        for pixel in range(len(c)):
            r = adjusted_array[column][pixel][0]
            g = adjusted_array[column][pixel][1]
            b = adjusted_array[column][pixel][2]
        
            adjusted_array[column][pixel][0] = (r * Rgain) + Roffset
            adjusted_array[column][pixel][1] = (g * Ggain) + Goffset
            adjusted_array[column][pixel][2] = (b * Bgain) + Boffset
    
    return adjusted_array"""

@njit
def adjust_pixels(image_array, Rgain, Ggain, Bgain, Roffset, Goffset, Boffset):
    adjusted_array = image_array.copy()  # Make a copy of the image array
    
    for column in range(len(adjusted_array)):
        for pixel in range(len(adjusted_array[column])):
            r, g, b = adjusted_array[column][pixel]
            
            r_adjusted = int((r * Rgain) + Roffset)
            g_adjusted = int((g * Ggain) + Goffset)
            b_adjusted = int((b * Bgain) + Boffset)
            
            # Clamp the adjusted values to the range of 0 to 255
            r_adjusted = max(0, min(r_adjusted, 255))
            g_adjusted = max(0, min(g_adjusted, 255))
            b_adjusted = max(0, min(b_adjusted, 255))
            
            adjusted_array[column][pixel] = [r_adjusted, g_adjusted, b_adjusted]
    
    return adjusted_array


frame_list = frame_list("overlap.csv")
im_dict, cropped_im_dict, plt_dict, overlap_dict, frame_dict, frame_overlap = make_dictionaries("U1556B", "overlap.csv")
#crop_overlaps(frame_dict, frame_overlap, im_dict)
image_stats = image_stats("frames")
gain_offsets = gain_offset(image_stats)
adjust = adjust_gain_offset(gain_offsets)



0
1
1
index      390-U1556B-19R-1-Pc7-1x.BMP_1B.jpg
Rmean                               79.475435
Bmean                               76.321175
Gmean                                 71.3113
Rstdv                               20.787519
Bstdv                               21.722417
Gstdv                               20.551428
Rgain                                       1
Ggain                                       1
Bgain                                       1
Roffset                                     0
Goffset                                     0
Boffset                                     0
Name: 0, dtype: object
index      390-U1556B-19R-1-Pc7-2x.BMP_2A.jpg
Rmean                                78.48283
Bmean                                74.32245
Gmean                               65.922308
Rstdv                               28.591665
Bstdv                               27.757692
Gstdv                               27.049967
Rgain                                        
Ggain

In [3]:
#run this code to normalize and save the original piece images 

"""values = range(3)
with tqdm(total=len(values), file=sys.stdout) as pbar:
    for i in range(len(adjust)):
        pbar.write('processed: %d' % (1 + i))
        pbar.update(1)
        sleep(1)
        for j in range(len(adjust.loc[i,"index"])):
            print(adjust.loc[i,"index"][j])
            image_array = np.array(cropped_im_dict[adjust.loc[i,"index"][j]])
            Rgain = adjust.loc[i,"Rgain"][j]
            Ggain = adjust.loc[i,"Ggain"][j]
            Bgain = adjust.loc[i,"Bgain"][j]
            Roffset = adjust.loc[i,"Roffset"][j]
            Goffset = adjust.loc[i,"Goffset"][j]
            Boffset = adjust.loc[i,"Boffset"][j]
            
            normalized_array = adjust_pixels(image_array, Rgain, Ggain, Bgain, Roffset, Goffset, Boffset)
        
            corrected_image = Image.fromarray(normalized_array)
            corrected_name = str(adjust.loc[i,"index"][j])#.replace(".BMP", ".jpg")
            corrected_image.save("normalized/"+corrected_name)"""

for i in range(len(adjust)):
    for j in range(len(adjust.loc[i,"index"])):
        print(adjust.loc[i,"index"][j])
        if "1x" in adjust.loc[i,"index"][j]: #skip 1x as the original is the master to not be replaced
            continue
            
        else:
            image_array = np.array(cropped_im_dict[adjust.loc[i,"index"][j]])
            Rgain = adjust.loc[i,"Rgain"][j]
            Ggain = adjust.loc[i,"Ggain"][j]
            Bgain = adjust.loc[i,"Bgain"][j]
            Roffset = adjust.loc[i,"Roffset"][j]
            Goffset = adjust.loc[i,"Goffset"][j]
            Boffset = adjust.loc[i,"Boffset"][j]

            normalized_array = adjust_pixels(image_array, Rgain, Ggain, Bgain, Roffset, Goffset, Boffset)

            corrected_image = Image.fromarray(normalized_array)
            corrected_name = str(adjust.loc[i,"index"][j])#.replace(".BMP", ".jpg")
            corrected_image.save("normalized/"+corrected_name)

390-U1556B-19R-1-Pc7-1x.BMP
390-U1556B-19R-1-Pc7-2x.BMP
390-U1556B-19R-1-Pc7-3x.BMP


In [9]:
image_stats



{'390-U1556B-9R-3-Pc1-2x.BMP_1B.jpg': {'Rmean': 43.56910123515628,
  'Bmean': 41.62014377060249,
  'Gmean': 41.76678978513841,
  'Rstdv': 9.896469755268331,
  'Bstdv': 10.308551107959493,
  'Gstdv': 9.428473019730678},
 '390-U1556B-9R-3-Pc1-3x.BMP_2A.jpg': {'Rmean': 61.292347891004106,
  'Bmean': 59.88031448301605,
  'Gmean': 56.29688316536021,
  'Rstdv': 17.738857656883397,
  'Bstdv': 16.943923187182538,
  'Gstdv': 15.210915119531226}}